### Import Libraries

In [32]:
from aps_toolkit import Auth
import requests
import os

### Get Token

In [2]:
auth = Auth()

In [12]:
token = auth.auth2leg()

In [13]:
URN = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02"

### Get Metadata

In [16]:
# get manifest 
URL = f"https://developer.api.autodesk.com/modelderivative/v2/designdata/{URN}/manifest"
# add headers authorization
headers = {
    "Authorization": f"Bearer {token.access_token}"
}
# request
response = requests.get(URL, headers=headers)
json_respone = response.json()

In [33]:
# filter children in json_respone have minmeType is "application/autodesk-db" in derivatives/children/children
childrens = json_respone['derivatives'][0]["children"]
path  = ""
for child in childrens:
    if child["type"] == "resource" and child["mime"] == "application/autodesk-db":
        path = child["urn"]
        break
print(path)
host = "https://developer.api.autodesk.com/"
# get download link
URL = f"{host}modelderivative/v2/designdata/{URN}/manifest/{path}"
print("URL: ", URL)
# request
response = requests.get(URL, headers=headers)
# save db file
dir = "./output"
if not os.path.exists(dir):
    os.makedirs(dir)
with open("./output/result.db", "wb") as f:
    f.write(response.content)
    f.close()
print("Downloaded file result.db")

urn:adsk.viewing:fs.file:dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02/output/Resource/model.sdb
URL:  https://developer.api.autodesk.com/modelderivative/v2/designdata/dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02/manifest/urn:adsk.viewing:fs.file:dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02/output/Resource/model.sdb
Downloaded file result.db


### Read SQLite file

In [35]:
# read sqlite file
import sqlite3
db_file = "./output/result.db"
try:
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    # Query the sqlite_master table to get the list of all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # Fetch the results
    tables = cursor.fetchall()

    # Extract table names and print them
    for table in tables:
        print(table[0])

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
finally:
    # Close the database connection when done
    conn.close()

_objects_id
_objects_attr
_objects_val
_objects_eav


### Object ID Reader

In [24]:
# Read data from _objects_attr
import pandas as pd
# Read data from _objects_id
conn = sqlite3.connect(db_file)
data_objects_id = pd.read_sql_query("SELECT * FROM _objects_id", conn)
data_objects_id.head(20)

,id,external_id,viewable_id
0,1,doc_d14ee89f-34c9-4d58-ae27-c584f63353df,None
1,2,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f7,None
2,3,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f8,None
3,4,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f9,None
4,5,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522fa,None
5,6,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522fb,None
6,7,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522fc,None
7,8,f9c62406-7c50-11d5-93b8-0000863f27ad-00001fdd,None
8,9,f9c62406-7c50-11d5-93b8-0000863f27ad-00001ff0,None
9,10,f9c62406-7c50-11d5-93b8-0000863f27ad-0000202d,None


### Object Attribute Reader

In [25]:
# Read data from _objects_attr
data_objects_id = pd.read_sql_query("SELECT * FROM _objects_attr", conn)
data_objects_id.head(20)

,id,name,category,data_type,data_type_context,description,display_name,flags,display_precision,forge_parameter
0,1,name,__name__,20,None,None,None,0,0,None
1,2,child,__child__,11,None,None,None,1,0,None
2,3,parent,__parent__,11,None,None,None,1,0,None
3,4,instanceof_objid,__instanceof__,11,None,None,None,1,0,None
4,5,hastable,__hastable__,11,None,None,None,1,0,None
5,6,viewable_in,__viewable_in__,20,None,None,None,1,0,None
6,7,xref,__externalref__,20,None,None,None,1,0,None
7,8,is_doc_property,__document__,1,None,None,None,3,0,None
8,9,schema_name,__document__,20,None,None,None,1,0,None
9,10,schema_version,__document__,20,None,None,None,1,0,None


### Object Value Reader

In [26]:
import pandas as pd
# Read data from _objects_val
data_objects_id = pd.read_sql_query("SELECT * FROM _objects_val", conn)
data_objects_id.head(20)

,id,value
0,1,Model
1,2,rvt
2,3,1.0
3,4,1
4,5,Revit Document
5,6,Exyte Group
6,7,"Design, Engineering & Construction"
7,8,
8,9,Issue Date
9,10,Project Status


### Object EAV Reader

In [27]:
# Read data from _objects_attr
import pandas as pd
# Read data from _objects_id
data_objects_id = pd.read_sql_query("SELECT * FROM _objects_eav", conn)
data_objects_id.head(20)

,id,entity_id,attribute_id,value_id
0,1,1,1,1
1,2,1,9,2
2,3,1,10,3
3,4,1,8,4
4,5,1,13,5
5,6,1,14,6
6,7,1,15,7
7,8,1,16,8
8,9,1,17,8
9,10,1,18,8


In [29]:
# Define and execute the SQL query with a JOIN statement
conn = sqlite3.connect(db_file)
# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id AS externalId, _objects_attr.category AS category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE category NOT LIKE '\_\_%\_\_' ESCAPE '\\' 
ORDER BY externalId;
"""
data = pd.read_sql_query(query, conn)
data.head(20)

,externalId,category,display_name,value
0,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Image,
1,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Keynote,
2,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,Export to IFC,By Type
3,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,Export to IFC As,
4,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,IFC Predefined Type,
5,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,IfcGUID,00ni1A4jzDhPGViPFEiZIK
6,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Model,
7,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Manufacturer,
8,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Comments,
9,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,URL,


### Get All Categories

In [30]:
# Query filter return list 
conn = sqlite3.connect(db_file)
# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id, _objects_attr.category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE _objects_attr.name LIKE '_RC';
"""
df_categories = pd.read_sql_query(query, conn)
# get unique values from the category column
categories = df_categories['value'].unique()
# create a dataframe for categories
df_categories = pd.DataFrame(categories, columns=['category'])
df_categories.head(50)
df_categories.sort_values(by=['category'], inplace=True)
df_categories.head(50)

,category
0,
35,<Sketch>
53,Air Terminals
5,Area Schemes
43,Automatic Sketch Dimensions
19,Building Type Settings
14,Cameras
61,Center Line
54,Center line
17,Color Fill Schema


In [31]:
# unormalized data with columns : external_id, RevitCategory, ParameterCategory, ParameterName, DataType, DataTypeContext,ParameterValue
conn = sqlite3.connect(db_file)
# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id, _objects_attr.category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE _objects_attr.name LIKE '_RC';
"""
df_categories = pd.read_sql_query(query, conn)
# get unique values from the category column
categories = df_categories['value'].unique()
categories

array(['', 'Materials', 'Elevations', 'Views', 'Primary Contours',
       'Area Schemes', 'Phases', 'Shared Site', 'Work Plane Grid',
       'Revision', 'Project Information', 'Structural Load Cases',
       'Sun Path', 'Internal Origin', 'Cameras', 'Section Boxes',
       'HVAC Zones', 'Color Fill Schema', 'HVAC Load Schedules',
       'Building Type Settings', 'Space Type Settings', 'Survey Point',
       'Project Base Point', 'Electrical Demand Factor Definitions',
       'Electrical Load Classifications',
       'Panel Schedule Templates - Branch Panel',
       'Panel Schedule Templates - Data Panel',
       'Panel Schedule Templates - Switchboard',
       'Electrical Load Classification Parameter Element',
       'Material Assets', 'Pipe Segments', 'Legend Components',
       'Generic Annotations', 'Lines', 'Detail Items', '<Sketch>',
       'Rooms', 'Spaces', 'Title Blocks', 'Schedules', 'Text Notes',
       'Dimensions', 'Constraints', 'Automatic Sketch Dimensions',
       'Revi